In [1]:
from Scripts.model import *

In [2]:
x = torch.ones((1, 1, 128, 128))

In [3]:
net = UNet(1)

In [5]:
y = net(x)

In [7]:
y.shape

torch.Size([1, 1, 128, 128])

In [3]:
enc = EncoderBlock(1, 2, padding=1)
y = enc(x)

In [4]:
y

tensor([[[[4.0122, 4.4350, 4.4350,  ..., 4.4350, 4.4350, 3.5100],
          [2.1633, 0.0288, 0.0288,  ..., 0.0288, 0.0288, 0.0330],
          [2.1633, 0.0288, 0.0288,  ..., 0.0288, 0.0288, 0.0330],
          ...,
          [2.1633, 0.0288, 0.0288,  ..., 0.0288, 0.0288, 0.0330],
          [2.1633, 0.0288, 0.0288,  ..., 0.0288, 0.0288, 0.0330],
          [1.3420, 0.0790, 0.0790,  ..., 0.0790, 0.0790, 0.0532]],

         [[2.7642, 5.2036, 5.2036,  ..., 5.2036, 5.2036, 5.5992],
          [0.7336, 0.0086, 0.0086,  ..., 0.0086, 0.0086, 0.1252],
          [0.7336, 0.0086, 0.0086,  ..., 0.0086, 0.0086, 0.1252],
          ...,
          [0.7336, 0.0086, 0.0086,  ..., 0.0086, 0.0086, 0.1252],
          [0.7336, 0.0086, 0.0086,  ..., 0.0086, 0.0086, 0.1252],
          [1.4177, 0.1045, 0.1045,  ..., 0.1045, 0.1045, 0.1564]]]],
       grad_fn=<MaxPool2DWithIndicesBackward0>)

In [5]:
y.shape

torch.Size([1, 2, 32, 32])

In [6]:
dec = DecoderBlock(2, 1, transpose_stride=2, out_padding=1)

x = dec(y, y)

In [7]:
x

tensor([[[[ 3.0452, -3.1293,  6.1165,  ..., -6.9095,  1.1591, -3.6795],
          [-2.9322, -4.5297, -3.2377,  ..., -0.4525,  1.5788, -0.2614],
          [ 0.0979, -1.1171, -1.5505,  ...,  0.2066, -1.4356,  2.0224],
          ...,
          [-0.1107,  0.0540,  0.8798,  ...,  0.1280,  0.1068,  0.0890],
          [ 0.0979, -0.5197, -0.4706,  ...,  0.0992,  0.0919,  0.1047],
          [ 0.0979,  0.0979,  0.0979,  ...,  0.0979,  0.0979,  0.0979]]]],
       grad_fn=<NativeBatchNormBackward0>)

In [8]:
x.shape

torch.Size([1, 1, 66, 66])